# Splitters
* Divide a large data asset in small parts.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 002-splitters.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **002-splitters**.

## Track operations
From now on, we can track the operations **and the cost** of this project from LangSmith:
* [smith.langchain.com](https://smith.langchain.com)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

## Reminder: steps of the RAG process.
* When you load a document, you end up with strings. Sometimes the strings will be too large to fit into the context window. In those occassions we will use the RAG technique:
    * **Split document in small chunks**.
    * Transform text chunks in numeric chunks (embeddings).
    * Load embeddings to a vector database (aka vector store).
    * Load question and retrieve the most relevant embeddings to respond it.
    * Sent the embeddings to the LLM to format the response properly.

## Splitters: divide the loaded document in small chunks of text
* Also called "Document Transformers".
* See the documentation page [here](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/).
* See the list of built-in splitters [here](https://python.langchain.com/v0.1/docs/integrations/document_transformers/).

#### Simple splitting by character: Character Splitter
* This splits based on characters (by default "\n\n") and measure chunk length by number of characters.
* The "Character Splitter" in the context of RAG (Retrieval-Augmented Generation) applications, specifically using LangChain's tools, is a method that divides text into smaller parts based on specific characters.
* By default, it uses double newline characters ("\n\n") to identify where one chunk of text ends and another begins.
* Each chunk is measured by its number of characters.
* This simple splitting method is useful in RAG applications to help manage and process large blocks of text by breaking them down into manageable, smaller pieces. This can enhance the efficiency and effectiveness of the text retrieval process, which is crucial in generating accurate and contextually relevant responses.

Here's a simple example to illustrate how the "Character Splitter" works in the context of RAG applications using the default delimiter ("\n\n").

#### Original Text:
```
Hello, welcome to our store!

\n\nWe offer a variety of products ranging from electronics to clothing.

\n\nOur store hours are 9 AM to 9 PM every day.

\n\nFeel free to ask for assistance if you need help finding anything.
```

#### After Applying Character Splitter:
1. **Chunk 1:**
   ```
   Hello, welcome to our store!
   ```

2. **Chunk 2:**
   ```
   We offer a variety of products ranging from electronics to clothing.
   ```

3. **Chunk 3:**
   ```
   Our store hours are 9 AM to 9 PM every day.
   ```

4. **Chunk 4:**
   ```
   Feel free to ask for assistance if you need help finding anything.
   ```

In this example, the text is split into four chunks based on the presence of "\n\n" between sections of text. Each chunk is a manageable size and clearly separated from the others, making it easier for a RAG system to handle and retrieve information from specific parts of the text as needed.

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/be-good.txt")

loaded_data = loader.load()

In [6]:
#loaded_data

In [7]:
#loaded_data[0].page_content

In [8]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [9]:
texts = text_splitter.create_documents([loaded_data[0].page_content])

In [10]:
len(texts)

2

In [11]:
texts[0]

Document(page_content='Be good')

In [12]:
# texts[1]

#### Splitting with metadata

In [13]:
metadatas = [{"chunk": 0}, {"chunk": 1}]

documents = text_splitter.create_documents(
    [loaded_data[0].page_content, loaded_data[0].page_content], 
    metadatas=metadatas
)

The `CharacterTextSplitter` from LangChain's text splitting utilities helps break down text into manageable chunks along with the option to attach metadata to each chunk. Here’s how it works in simple terms:

1. **Initialization of the Splitter**: The splitter is configured with several parameters:
   - `separator`: This is the character or string at which the text will be split. In this example, it’s set to `"\n\n"`, which means the text will be divided at every occurrence of double newlines.
   - `chunk_size`: This determines the maximum size of each chunk in characters. Here, each chunk can be up to 1000 characters long.
   - `chunk_overlap`: This allows chunks to overlap by a specified number of characters, set here to 200. Overlapping helps ensure that no crucial information is cut off awkwardly at the end of a chunk.
   - `length_function`: This is typically a function to measure the length of the text; `len` simply counts the number of characters.
   - `is_separator_regex`: This indicates whether the separator is a regular expression or not. Here, it’s `False`, meaning the separator is treated as a literal string.

2. **Creating Chunks**: The `create_documents` method is used to split the text. It takes in the text (or texts) and optional metadata, and splits the text based on the defined parameters.

3. **Attaching Metadata**: Optionally, metadata can be attached to each chunk to provide additional context or identifiers. For example, metadata might tag each chunk with its sequence number or categorize it based on content. In the example, metadata like `{"chunk": 0}` could signify that it’s the first chunk.

#### Example Code Explanation:
- **First Splitter Use**: Initially, the splitter is used to split the content of `loaded_data[0].page_content` without metadata. This will just divide the text based on the provided settings.
- **Second Splitter Use**: The same text is then split again, but this time with metadata provided (`metadatas=[{"chunk": 0}, {"chunk": 1}]`). This second run distinguishes each chunk with additional information.

#### Example Outputs:
- **Number of Chunks**: This would typically be printed using `len(texts)`, showing how many chunks the text was split into.
- **First Chunk**: By printing `texts[0]`, you would see the content of the first chunk.
- **First Chunk with Metadata**: Printing `documents[0]` after the second splitting operation would show the first chunk of text along with its corresponding metadata, illustrating how metadata is associated with text chunks.

This method is particularly useful in applications where chunks of text need to be processed independently, but still require some form of contextual or sequential linking.

In [14]:
documents[0]

Document(metadata={'chunk': 0}, page_content='Be good')

In [15]:
print(documents[0])

page_content='Be good' metadata={'chunk': 0}


## Recursive Character Splitter
* This text splitter is the recommended one for generic text. 
* It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. 
* This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

#### Simple Explanation:

* The "Recursive Character Splitter" is a method used to divide text into smaller, more manageable chunks, designed specifically to maintain the semantic integrity of the text.
* It operates by attempting to split the text using a list of characters in a specified order—beginning with the largest units like paragraphs, then moving to sentences, and finally to individual words if needed.
* The default sequence for splitting is ["\n\n", "\n", " ", ""], which means it first tries to split the text at double newline characters to separate paragraphs, then at single newlines for any remaining large blocks, followed by spaces to isolate sentences or phrases, and finally using an empty string if finer splitting is necessary.
* This method is particularly effective because it tries to keep text chunks as meaningful and complete as possible, ensuring that each chunk has a coherent piece of information.

#### Example of Use:

#### Original Text:
```
Hello, welcome to our store!

\n\nWe offer a variety of products. Our range includes electronics, clothing, and home appliances.
\nOur staff is available to help you during store hours: 9 AM to 9 PM every day.
```

#### Applying Recursive Character Splitter:
- First attempt with `"\n\n"`:
  1. **Chunk 1:** `Hello, welcome to our store!`
  2. **Chunk 2:** `We offer a variety of products. Our range includes electronics, clothing, and home appliances.\nOur staff is available to help you during store hours: 9 AM to 9 PM every day.`

- Second attempt with `"\n"` for remaining long chunk:
  1. **New Chunk 2:** `We offer a variety of products. Our range includes electronics, clothing, and home appliances.`
  2. **Chunk 3:** `Our staff is available to help you during store hours: 9 AM to 9 PM every day.`

- No further splits are necessary as all chunks are now of manageable size.

In this example, the text is initially split into two chunks using the double newline. Since one chunk is still quite long, it then uses a single newline to split it further. Each chunk retains coherent, complete information, reflecting the effective use of the recursive character splitter to preserve the semantic structure of the text.

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=26,
    chunk_overlap=4
)

In [18]:
text1 = 'abcdefghijklmnopqrstuvwxyzabcdefg'

In [19]:
text2 = """
Data that Speak
LLM Applications are revolutionizing industries such as 
banking, healthcare, insurance, education, legal, tourism, 
construction, logistics, marketing, sales, customer service, 
and even public administration.

The aim of our programs is for students to learn how to 
create LLM Applications in the context of a business, 
which presents a set of challenges that are important 
to consider in advance.
"""

In [20]:
recursive_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [21]:
recursive_splitter.split_text(text2)

['Data that Speak',
 'LLM Applications are',
 'are revolutionizing',
 'industries such as',
 'banking, healthcare,',
 'insurance, education,',
 'legal, tourism,',
 'construction, logistics,',
 'marketing, sales,',
 'customer service,',
 'and even public',
 'administration.',
 'The aim of our programs',
 'is for students to learn',
 'how to',
 'create LLM Applications',
 'in the context of a',
 'a business,',
 'which presents a set of',
 'of challenges that are',
 'are important',
 'to consider in advance.']

In [22]:
second_recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

The separators `["\n\n", "\n", "(?<=\. )", " ", ""]` refer to a sequence of characters and patterns used to split text into smaller parts, with each separator serving a specific function based on the structure of text:

1. `"\n\n"` - This separator targets double newlines, which are often used to denote separate paragraphs or sections within a text. Splitting here is intended to separate distinct thematic or topical blocks of content.

2. `"\n"` - This targets single newline characters, which typically indicate a new line within a paragraph or a soft break in the content, such as between list items or sub-paragraphs.

3. `"(?<=\. )"` - This is a regular expression that looks for a period followed by a space, typically used to signify the end of a sentence. The `(?<= )` part is a "lookbehind" assertion in regex, which means it checks for the occurrence of a period and space before splitting, but does not include them in the split, thus keeping sentences intact.

4. `" "` - This targets single space characters, which are commonly used to separate words. Splitting on spaces can break down text into individual words or phrases, especially useful when finer granularity is needed.

5. `""` - An empty string as a separator is used in text processing to split a text into its individual characters, essentially decomposing the text down to its most basic elements.

Each of these separators is used to progressively split the text into smaller and smaller chunks, starting from larger structural divisions like paragraphs and moving down to the level of individual characters, depending on the needs of the application.

In [23]:
second_recursive_splitter.split_text(text2)

['Data that Speak\nLLM Applications are revolutionizing industries such as \nbanking, healthcare, insurance, education, legal, tourism,',
 'construction, logistics, marketing, sales, customer service, \nand even public administration.',
 'The aim of our programs is for students to learn how to \ncreate LLM Applications in the context of a business,',
 'which presents a set of challenges that are important \nto consider in advance.']

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 002-splitters.py
* In terminal, make sure you are in the directory of the file and run:
    * python 002-splitters.py